In [1]:
pip install pyspark

  Using cached pyspark-3.5.0-py2.py3-none-any.whl


In [2]:
pip install findspark

In [3]:
# Import findspark and initialize.
import findspark
findspark.init()

In [4]:
# Import packages
from pyspark.sql import SparkSession
import time
import urllib

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [5]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

In [6]:
#spark.addfile(f)
spark.sparkContext.addFile(url)

In [7]:
# 2. Create a temporary view of the DataFrame.
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

In [8]:
home_sales_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)



In [9]:
home_sales_df.createOrReplaceTempView("home_sales")


In [10]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
query = """
SELECT
    YEAR(date) AS year,
    ROUND(AVG(price), 2) AS avg_price
FROM
    home_sales
WHERE
    bedrooms = 4
GROUP BY
    YEAR(date)
ORDER BY
    year
"""

result_df = spark.sql(query)
result_df.show()


+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [11]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
b = """
SELECT
    date_built,
    ROUND(AVG(price), 2) AS avg_price
FROM
    home_sales
WHERE
    bedrooms = 3 AND bathrooms = 3
GROUP BY
    date_built
ORDER BY
    date_built
"""

result_df = spark.sql(b)
result_df.show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [12]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
c = """
SELECT
    date_built,
    ROUND(AVG(price), 2) AS avg_price
FROM
    home_sales
WHERE
    bedrooms = 3 AND
    bathrooms = 3 AND
    floors = 2 AND
    sqft_living >= 2000
GROUP BY
    date_built
ORDER BY
    date_built
"""

result_df = spark.sql(c)
result_df.show()



+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [34]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.
from pyspark.sql import functions as F
from time import time

# Start time
start_time = time()

# SQL Query
d = """
SELECT
    view, ROUND(AVG(price), 2) AS avg_price
FROM
    home_sales
WHERE
    price >= 350000

GROUP BY
    view
"""

# Execute the query
result_df = spark.sql(d)
result_df.show()

# End time
end_time = time()

# Calculate and print the runtime
runtime = end_time - start_time
print(f"Query Runtime: {runtime} seconds")

+----+----------+
|view| avg_price|
+----+----------+
|  31| 399856.95|
|  85|1056336.74|
|  65| 736679.93|
|  53|  755214.8|
|  78|1080649.37|
|  34| 401419.75|
|  81|1053472.79|
|  28| 402124.62|
|  76|1058802.78|
|  26| 401506.97|
|  27| 399537.66|
|  44| 400598.05|
|  12| 401501.32|
|  91|1137372.73|
|  22| 402022.68|
|  93|1026006.06|
|  47|  398447.5|
|   1| 401044.25|
|  52| 733780.26|
|  13| 398917.98|
+----+----------+
only showing top 20 rows

Query Runtime: 0.5278639793395996 seconds


In [14]:
# 7. Cache the the temporary table home_sales.
home_sales_df.cache()

DataFrame[id: string, date: date, date_built: int, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]

In [15]:
# 8. Check if the table is cached.
spark.catalog.listTables()

[Table(name='home_sales', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [35]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

from time import time

# Query
e = """
SELECT
    view, ROUND(AVG(price), 2) AS avg_price
FROM
    home_sales
WHERE
    price >= 350000

GROUP BY
    view
"""

# Measure runtime for uncached data
start_time_uncached = time()
spark.sql(e).show()
end_time_uncached = time()

runtime_uncached = end_time_uncached - start_time_uncached
print(f"Uncached Query Runtime: {runtime_uncached} seconds")


+----+----------+
|view| avg_price|
+----+----------+
|  31| 399856.95|
|  85|1056336.74|
|  65| 736679.93|
|  53|  755214.8|
|  78|1080649.37|
|  34| 401419.75|
|  81|1053472.79|
|  28| 402124.62|
|  76|1058802.78|
|  26| 401506.97|
|  27| 399537.66|
|  44| 400598.05|
|  12| 401501.32|
|  91|1137372.73|
|  22| 402022.68|
|  93|1026006.06|
|  47|  398447.5|
|   1| 401044.25|
|  52| 733780.26|
|  13| 398917.98|
+----+----------+
only showing top 20 rows

Uncached Query Runtime: 0.7443878650665283 seconds


In [17]:
# Cache the DataFrame
home_sales_df.cache()

# Perform an action to trigger caching
home_sales_df.count()  # This is just to trigger caching


33287

In [18]:
# Measure runtime for cached data
start_time_cached = time()
spark.sql(query).show()
end_time_cached = time()

runtime_cached = end_time_cached - start_time_cached
print(f"Cached Query Runtime: {runtime_cached} seconds")


+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+

Cached Query Runtime: 0.8330254554748535 seconds


In [19]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_df.write.partitionBy('date_built').parquet('p_home_sales',mode='overwrite')

In [20]:
output_path = "/content/data/partitioned_data"

# Write the DataFrame to Parquet format, partitioned by 'date_built'
home_sales_df.write.partitionBy("date_built").parquet(output_path)

In [21]:
# 11. Read the formatted parquet data.
input_path = "/content/data/partitioned_data"

# Read the Parquet data into a DataFrame
partitioned_home_sales_df = spark.read.parquet(input_path)

# Show some data to verify it's been read correctly
partitioned_home_sales_df.show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [22]:
# 12. Create a temporary table for the parquet data.
# Create a temporary table
partitioned_home_sales_df.createOrReplaceTempView("temp_home_sales_table")

# Now you can use SQL queries against this table
# For example, to display the first few rows of the table
spark.sql("SELECT * FROM temp_home_sales_table LIMIT 10").show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [36]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

from time import time

# SQL Query
query = """
SELECT
    view, ROUND(AVG(price), 2) AS avg_price
FROM
    home_sales
WHERE
    price >= 350000

GROUP BY
    view
"""

# Measure runtime for Parquet DataFrame
start_time_parquet = time()
spark.sql(query).show()
end_time_parquet = time()

runtime_parquet = end_time_parquet - start_time_parquet
print(f"Parquet DataFrame Query Runtime: {runtime_parquet} seconds")


+----+----------+
|view| avg_price|
+----+----------+
|  31| 399856.95|
|  85|1056336.74|
|  65| 736679.93|
|  53|  755214.8|
|  78|1080649.37|
|  34| 401419.75|
|  81|1053472.79|
|  28| 402124.62|
|  76|1058802.78|
|  26| 401506.97|
|  27| 399537.66|
|  44| 400598.05|
|  12| 401501.32|
|  91|1137372.73|
|  22| 402022.68|
|  93|1026006.06|
|  47|  398447.5|
|   1| 401044.25|
|  52| 733780.26|
|  13| 398917.98|
+----+----------+
only showing top 20 rows

Parquet DataFrame Query Runtime: 0.7617874145507812 seconds


In [24]:
home_sales_df.cache()
home_sales_df.createOrReplaceTempView("cached_home_sales")

# Measure runtime for cached DataFrame
start_time_cached = time()
spark.sql(query).show()
end_time_cached = time()

runtime_cached = end_time_cached - start_time_cached
print(f"Cached DataFrame Query Runtime: {runtime_cached} seconds")


+---------------+
|avg_view_rating|
+---------------+
|          32.26|
+---------------+

Cached DataFrame Query Runtime: 0.37348246574401855 seconds


In [25]:
# 14. Uncache the home_sales temporary table.
home_sales_df.unpersist()

DataFrame[id: string, date: date, date_built: int, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]

In [26]:
# 15. Check if the home_sales is no longer cached
is_cached = home_sales_df.is_cached
print(f"Is home_sales_df cached: {is_cached}")

Is home_sales_df cached: False
